## Using an E3NN network to predict electron densities

In this tutorial we show how an E3NN network can be used to predict electron densities. One reason this might be a good idea is that electron densities can be represented in a spherical harmonic basis on atom centers. This fits naturally with the E3NN framework. 


In [1]:
import numpy as np
import pickle
import torch
import random
from functools import partial

from e3nn.kernel import Kernel
from e3nn.point.operations import Convolution
from e3nn.non_linearities import GatedBlock
from e3nn.non_linearities import rescaled_act
from e3nn.non_linearities.rescaled_act import relu, sigmoid
from e3nn.radial import CosineBasisModel
from e3nn.radial import GaussianRadialModel

torch.set_default_dtype(torch.float64)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

First we load the data. I have saved this in a pickle.
For this particular example the dataset is ~6000 water dimer structures with density represented in "a2" density fitting basis set.

Oxygen: 8s, 4p, 4d 

Hydrogen: 4s, 1p, 1d

In [2]:
## load density data
picklename = "./density_data/dimer_data.pckl"
with open(picklename, 'rb') as f:
    dataset_coeffs, dataset_onehot, dataset_geom, dataset_typemap, Rs_out_list, coeff_by_type = pickle.load(f)


### Step 1: Define a model

The thing that makes this task tricky is predicting different numbers of spherical harmonics on each atomic center. To address this problem, we introduce a class, Mixer, to handle this.

Our network consists of 3 layers.
1. Input layer: Geometry and one-hot atom encoding 
2. Middle layer: Rs = [(2,0),(1,1),(1,2)] (Up to L=2) Convolution + GatedBlock nonlinearity
3. Final layer: 2 types of convolution, 1 for oxygen and 1 for hydrogen

In [3]:
## define model

class Mixer(torch.nn.Module):
    def __init__(self, Op, Rs_in_s, Rs_out):
        super().__init__()
        self.ops = torch.nn.ModuleList([
            Op(Rs_in, Rs_out)
            for Rs_in in Rs_in_s
        ])

    def forward(self, *args, n_norm=1):
        # It simply sums the different outputs
        y = 0
        for m, x in zip(self.ops, args):
            y += m(*x, n_norm=n_norm)
        return y


class Network(torch.nn.Module):
    def __init__(self, Rs_in, Rs_out_list, max_radius=3.0, number_of_basis=3, radial_layers=3, basistype="Gaussian"):
        super().__init__()

        #sp = rescaled_act.Softplus(beta=5)
        #sp = rescaled_act.ShiftedSoftplus(beta=5)
        sp = torch.nn.Tanh()

        # the [0] is just to get first_layer in stripped form.
        # will not work for Rs_in with more than L=0
        first_layer = Rs_in[0]
        last_shared_layer = (2,1,1)

        representations = [first_layer, last_shared_layer]
        representations = [[(mul, l) for l, mul in enumerate(rs)] for rs in representations]

        if (basistype == 'Gaussian'):
            rad_basis = GaussianRadialModel
        elif (basistype == 'Cosine'):
            rad_basis = CosineBasisModel
        else:
            print ("Only Gaussian and Cosine Radial basis are currently supported")

        RadialModel = partial(rad_basis, max_radius=max_radius,
                              number_of_basis=number_of_basis, h=100,
                              L=radial_layers, act=sp)

        K = partial(Kernel, RadialModel=RadialModel)
        C = partial(Convolution, K)
        M = partial(Mixer, C)  # wrap C to accept many input types

        def make_layer(Rs_in, Rs_out):
            act = GatedBlock(Rs_out, sp, sigmoid)
            conv = Convolution(K, Rs_in, act.Rs_in)
            return torch.nn.ModuleList([conv, act])

        self.layers = torch.nn.ModuleList([
            make_layer(Rs_layer_in,Rs_layer_out)
            for Rs_layer_in, Rs_layer_out in zip(representations[:-1], representations[1:])
        ])

        ## set up the split final layer
        m = []
        for rs in Rs_out_list:
            m.append(M([representations[-1], representations[-1]], rs))
        
        # final layer is indexed in order of atom type
        self.final_layer = torch.nn.ModuleList([
            m[i] for i in range(len(m))
        ])

    def forward(self, input, geometry, atom_type_map):
        output = input
        batch, N, _ = geometry.shape

        for conv, act in self.layers:
            output = conv(output, geometry, n_norm=N)
            output = act(output)

        ## split final layer
        geometry_list = []
        feature_list = []
        for i, item in enumerate(atom_type_map):
            geometry_list.append(geometry[0][item])
            feature_list.append(output[0][item])

        ## this is assuming that there are only two atom types!
        ## it should work, though for any arbitrary order of O and H in xyzfile!
        featuresO = feature_list[0].unsqueeze(0)
        featuresH = feature_list[1].unsqueeze(0)
        geometryO = geometry_list[0].unsqueeze(0)
        geometryH = geometry_list[1].unsqueeze(0)
        
        final_layer_output = []
        for i, layer in enumerate(self.final_layer):
            if (i == 0):
                final = layer((featuresO, geometryO, geometryO), (featuresH, geometryH, geometryO), n_norm = N)
            if (i == 1):
                final = layer((featuresO, geometryO, geometryH), (featuresH, geometryH, geometryH), n_norm = N) 
            final_layer_output.append(final)

        # return list of outputO and outputH
        output = final_layer_output

        return output



### Step 2: Initilize th network

Let's initialize a rough model. Here's a brief description of the parameters:
- max_radius: Distance of the center of the farthest radial function from convolution center
- num_basis: Number of radial basis functions to use; more=finer grain detail
- radial_layers: Number of layers in radial basis network (number of nonlinearity operations)
- basistype: What type of functions to use for radial basis; default is Gaussians

We pass these parameters in as a dictionary so that we can save them for later use if we want to save the model.

Then we send the model to the GPU

The output shows us a helpful schematic of what kinds of operations our network is going to use.


In [4]:
## set arguments to network
maxradius = 3.0
numbasis = 20
radiallayers = 3
radialbasis = "Gaussian"
## set Rs_in based on onehot vector
Rs_in = [(len(dataset_typemap[0]),0)]
mydict = {"Rs_in":Rs_in, "Rs_out_list":(Rs_out_list), "max_radius":maxradius,
            "number_of_basis":numbasis, "radial_layers":radiallayers, 
            "basistype":radialbasis}

net = Network(**mydict)

net.to(device)

Network(
  (layers): ModuleList(
    (0): ModuleList(
      (0): Convolution(
        (kernel): Kernel (2x0 -> 2x0,1,2,2x0)
      )
      (1): GatedBlock(
        (scalar_act): Tanh()
      )
    )
  )
  (final_layer): ModuleList(
    (0): Mixer(
      (ops): ModuleList(
        (0): Convolution(
          (kernel): Kernel (2x0,1,2 -> 8x0,4x1,4x2)
        )
        (1): Convolution(
          (kernel): Kernel (2x0,1,2 -> 8x0,4x1,4x2)
        )
      )
    )
    (1): Mixer(
      (ops): ModuleList(
        (0): Convolution(
          (kernel): Kernel (2x0,1,2 -> 4x0,1,2)
        )
        (1): Convolution(
          (kernel): Kernel (2x0,1,2 -> 4x0,1,2)
        )
      )
    )
  )
)

### Step 3: Set up training

From here, training the model looks virtually identical to any other training one might do with a typical neural network in pytorch. In this case we are going to use the Adam optimizer and minibatches.

In [5]:
## set up training

net.train()

optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)
optimizer.zero_grad()
loss_fn = torch.nn.modules.loss.MSELoss()

max_steps = 2000
minibatch_size = 16


### Step 4: Train and evaluate Test error

In [6]:
loss_minibatch = 0
for step in range(max_steps):
    i = random.randint(0, len(dataset_geom) - 3001)

    onehot = dataset_onehot[i]
    points = dataset_geom[i]
    atom_type_map = dataset_typemap[i]
    coeffs = dataset_coeffs[i]

    outputO, outputH = net(onehot.to('cuda'),points.to('cuda'),atom_type_map)
    outputO = torch.flatten(outputO)
    outputH = torch.flatten(outputH)
    output = torch.cat((outputO,outputH),0).view(1,1,-1)

    loss = loss_fn(output.to('cuda'), coeffs.to('cuda'))
    step_loss = loss.item()
    loss.backward()
    loss_minibatch += step_loss

    if (step+1)%minibatch_size == 0:
        optimizer.step()
        optimizer.zero_grad()
        loss_minibatch = 0

    if step % 100 == 0:
        print('\nStep {0}, Loss {1}'.format(step, step_loss))
        j = random.randint(3000, len(dataset_geom) - 1)

        onehot = dataset_onehot[j]
        points = dataset_geom[j]*3
        atom_type_map = dataset_typemap[j]
        coeffs = dataset_coeffs[j]

        outputO, outputH = net(onehot.to('cuda'),points.to('cuda'),atom_type_map)
        outputO = torch.flatten(outputO)
        outputH = torch.flatten(outputH)
        output = torch.cat((outputO,outputH),0).view(1,1,-1)

        loss = loss_fn(output.to('cuda'), coeffs.to('cuda'))
        print('\nTest Loss {0}'.format(loss.item()))




Step 0, Loss 0.11907389512059059

Test Loss 0.11676265633713862

Step 100, Loss 0.03316797714839528

Test Loss 0.042259300544035296

Step 200, Loss 0.006693016660111862

Test Loss 0.03715066012161617

Step 300, Loss 0.002286758580877273

Test Loss 0.02890152602698776

Step 400, Loss 0.000838089295964252

Test Loss 0.03771684601099253

Step 500, Loss 0.0013117253601525177

Test Loss 0.03602544925479344

Step 600, Loss 0.0005144868588334724

Test Loss 0.03482248736254372

Step 700, Loss 0.00041751631888181407

Test Loss 0.034544754305458246

Step 800, Loss 0.0003568325774010871

Test Loss 0.03817526198501094

Step 900, Loss 0.0001873451335784625

Test Loss 0.03503706028378532

Step 1000, Loss 6.141958215722778e-05

Test Loss 0.03443993417625504

Step 1100, Loss 0.00011824445958179925

Test Loss 0.03423613068594088

Step 1200, Loss 8.411632141934517e-05

Test Loss 0.037264724659576864

Step 1300, Loss 5.415781352633813e-05

Test Loss 0.035049781645635346

Step 1400, Loss 6.68669554964026

In [11]:
from density_analysis_utils import *

testnumelectrons(net,10,"./density_data/a2.gbs",dataset_onehot,dataset_geom,dataset_typemap,coeff_by_type)


Now testing number of electrons on 10 randomly selected dimers

Number of electrons for structure 4244:
   True: 20.01397367619595
     ML: 20.168971621791776

Number of electrons for structure 5544:
   True: 20.013671760120452
     ML: 20.37076163987278

Number of electrons for structure 4307:
   True: 20.01374622715632
     ML: 19.84391153206681

Number of electrons for structure 5401:
   True: 20.013734417529992
     ML: 19.92265841669717

Number of electrons for structure 4539:
   True: 20.012783336427013
     ML: 20.142424568600326

Number of electrons for structure 5015:
   True: 20.013881680878303
     ML: 20.54106900971555

Number of electrons for structure 3700:
   True: 20.01415385491924
     ML: 19.293529626836648

Number of electrons for structure 4046:
   True: 20.014699970000358
     ML: 19.786969186866177

Number of electrons for structure 4177:
   True: 20.01361603500072
     ML: 20.14281878417792

Number of electrons for structure 3158:
   True: 20.01407317479071
    